### 데이터 불러오기

In [ ]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

In [ ]:
df = pd.read_csv('Regression_data_preprocessing.csv')
target = 'Rings'
y = df[target]
x = df.drop(target, axis =1)

### 1. 배치 정규화 - 사용 X
배치 정규화는 일반적으로 분류(classification) 문제에서 성능 향상에 더 많이 사용됩니다. 이는 배치 정규화가 입력 데이터의 분포를 정규화하고, 이를 통해 다음 층으로 전달되는 값을 안정화시키기 때문입니다.

### 2. Dropout - 성능에 따라 사용
드롭아웃은 분류(classification) 문제에서 주로 사용되는 정규화 방법이지만, 회귀(regression) 문제에서도 일부 경우에 사용될 수 있습니다. 회귀 모델에서 드롭아웃을 사용하면, 모델이 특정 입력값에 과도하게 의존하는 것을 방지하여 일반화 성능을 향상시킬 수 있습니다.

### 활성화 함수
1. Relu:
 입력층(input layer)과 은닉층(hidden layer)에서는 일반적으로 ReLU(Rectified Linear Unit) 활성화 함수가 많이 사용됩니다. ReLU 함수는 계산이 간단하고, 학습 속도가 빠르며, 특정 입력값에 대해 미분 가능하기 때문에, 딥러닝 모델에서 가장 많이 사용되는 활성화 함수 중 하나입니다.

2. tanh:
 입력값의 범위가 -1에서 1 사이인 경우에는 하이퍼볼릭 탄젠트(tanh) 함수를 사용하는 것이 좋습니다. 이는 입력값이 크거나 작을 때 출력값이 포화되는 문제를 해결할 수 있기 때문입니다. 따라서 입력값의 범위에 따라 적절한 활성화 함수를 선택하는 것이 좋습니다.

3. sigmoid:
 입력값을 0과 1 사이의 값으로 변환하는 함수로, 이진 분류(binary classification) 문제에서 출력층의 활성화 함수로 많이 사용됩니다. 하지만 회귀(regression) 문제에서는 선형 모델과 달리 출력값이 제한되지 않아야 하므로, sigmoid 함수를 활성화 함수로 사용하는 것은 적합하지 않습니다.

 4. linear:
 회귀 문제에서는 출력층의 활성화 함수로 linear 함수를 사용하는 것이 적합합니다. linear 함수는 입력값과 동일한 값을 출력하므로, 출력값의 범위가 제한되지 않습니다. 따라서 linear 함수를 사용하면 모델이 임의의 값을 예측할 수 있습니다.

In [ ]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")

# 표준화 진행 여부
if input_1 == 'y':
    scaler = StandardScaler()
    X = scaler.fit_transform(x)
    print("[안내] 데이터 표준화를 진행했습니다.")
else:
    X = x
    print("[안내] 데이터 표준화를 진행하지 않습니다.")

# 검증 데이터 진행 여부
if input_2 == 'y':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 추가로 분리했습니다.")

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 분리하지 않았습니다.")



[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.


### 모델 구현

1. SGD (Stochastic Gradient Descent):
가장 기본적인 옵티마이저로, 경사 하강법의 확률적인 버전입니다.
각 학습 단계에서 미니 배치(mini-batch) 단위로 데이터를 사용하여 가중치를 업데이트합니다.
단순하고 직관적인 방법이지만, 수렴 속도가 느리고 지역 최소값(local minimum)에 빠질 가능성이 있습니다.

2. Adam (Adaptive Moment Estimation):
학습률(learning rate)을 조정하는 방법을 통해 경사 하강법을 개선한 알고리즘입니다.
학습 속도를 개선하기 위해 모멘텀(Momentum)과 학습률 스케줄링(learning rate scheduling)을 조합합니다.
이동 평균(moving average)을 사용하여 각 가중치의 업데이트 속도를 조절하며, 자동으로 적응적인 학습률을 제공합니다.
다양한 유형의 신경망 구조와 데이터에 대해 일반적으로 좋은 성능을 보입니다.

3. RMSProp (Root Mean Square Propagation):
과거 그래디언트(gradient)의 제곱을 이동 평균하여 학습률을 조정하는 알고리즘입니다.
최근 그래디언트에 더 큰 가중치를 부여하여 중요한 그래디언트를 잘 반영합니다.
이동 평균을 사용하여 각 가중치의 업데이트 속도를 조절하며, 최적의 학습률을 자동으로 조정합니다.
비교적 안정적인 학습을 제공하고, RNN(Recurrent Neural Network)과 같은 모델에서 잘 작동하는 경향이 있습니다.

### 정확도 
- custom metric 사용 : 절대 비율 오차 (baseline)

In [ ]:
# method_custom_metric 구현
def accuracy(y_true, y_pred):
    return 1 - tf.abs((y_true - y_pred) / y_true) 

In [ ]:
# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용)
act_func = ['relu', 'tanh']
batch_lst = [8, 16, 32, 64, 128]
opt_lst = ['adam', 'rmsprop', 'sgd']
best_accuracy = 0.0
best_hyperparams = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(64, activation=func, input_dim=x.shape[1]))
            model.add(Dropout(0.1))  # Dropout 추가
            model.add(Dense(32, activation=func))
            model.add(Dropout(0.1))  # Dropout 추가               
            model.add(Dense(16, activation=func))
            model.add(Dropout(0.1))  # Dropout 추가
            model.add(Dense(8, activation=func))
            model.add(Dropout(0.1))  # Dropout 추가
            model.add(Dense(4, activation=func))
            model.add(Dropout(0.1))  # Dropout 추가
            model.add(Dense(1, activation='linear'))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=opti, metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='accuracy', patience=5)
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'optimizer': opti}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)



In [ ]:
# 최고의 모델 찾기 - 검증 데이터와 표준화 진행한 데이터로 성능 구현(dropout사용X)
act_func = ['relu', 'tanh']
batch_lst = [8, 16, 32, 64, 128]
opt_lst = ['adam', 'rmsprop', 'sgd']
best_accuracy = 0.0
best_hyperparams = {}

for func in act_func:
    for batch in batch_lst:
        for opti in opt_lst:
            # 모델 구현
            model = Sequential()
            model.add(Dense(64, activation=func, input_dim=x.shape[1]))
            model.add(Dense(32, activation=func))              
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(4, activation=func))
            model.add(Dense(1, activation='linear'))

            # 모델 컴파일
            model.compile(loss='mse', optimizer=opti, metrics=[accuracy])

            # early stopping 구현 - 커스텀 정확도 기준
            early_stopping = EarlyStopping(monitor='accuracy', patience=5)
            model.fit(X_train, y_train, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping])
            loss, acc = model.evaluate(X_val, y_val, verbose=2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'batch_size': batch, 'optimizer': opti, 'dropout': 'no'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)



### BEST MODEL
1. early stopping을 통해 정확도가 높은 모델 선정

2. 검증 데이터를 통해 검증 정확도가 높은 모델 선정 - 과적합 방지

In [ ]:
print('Best hyperparameters:', best_hyperparams)
print('Best accuracy:', best_accuracy)

Best hyperparameters: {'activation': 'tanh', 'batch_size': 32, 'optimizer': 'sgd', 'dropout': 'no'}
Best accuracy: 0.8571187257766724


In [ ]:
# 최고 모델 사용자 친화적 구현
start_time = time.time()
print("[안내] 모델이 실행됩니다.")
model = Sequential()
model.add(Dense(64, activation='tanh', input_dim=x.shape[1]))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='sgd', metrics=[accuracy])
early_stopping = EarlyStopping(monitor='accuracy', patience=5)

if input_2 == 'y':
    model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])
    y_pred = model.predict(X)
else:
    model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
    y_pred = model.predict(X)

print("[안내] 최종 모델")
if input_2 == 'y':
    print("[안내] train loss, accuracy")
    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
    print("[안내] validation loss, accuracy")
    loss, acc = model.evaluate(X_val, y_val, verbose=2)
else:
    print("[안내] train loss, accuracy")
    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
    print("[안내] test loss, accuracy")
    loss, acc = model.evaluate(X_test, y_test, verbose=2)

end_time = time.time()

execution_time = end_time - start_time
print("[안내] 실행 시간 : {:.3f} seconds".format(execution_time))

input_3 = input("[안내] 예측 샘플을 확인할까요? : ")
if input_3 == 'y':
    print("[안내] 샘플 10개의 결과")
    new_y = y
    stacked_array = np.vstack((y_pred))
    new_df = pd.DataFrame(stacked_array)
    new_y = pd.DataFrame(new_y)
    new_y['pred'] = new_df[0]
    print(new_y.sample(10))
else:
    print("[안내] 실행을 종료합니다.")

[안내] 모델이 실행됩니다.
Epoch 1/1000
84/84 [==============================] - 1s 4ms/step - loss: 13.5313 - accuracy: 0.7575 - val_loss: 7.3383 - val_accuracy: 0.7576
Epoch 2/1000
84/84 [==============================] - 0s 2ms/step - loss: 6.2903 - accuracy: 0.8226 - val_loss: 6.3149 - val_accuracy: 0.8223
Epoch 3/1000
84/84 [==============================] - 0s 2ms/step - loss: 5.4810 - accuracy: 0.8344 - val_loss: 4.9085 - val_accuracy: 0.8352
Epoch 4/1000
84/84 [==============================] - 0s 2ms/step - loss: 5.2057 - accuracy: 0.8381 - val_loss: 5.2294 - val_accuracy: 0.8051
Epoch 5/1000
84/84 [==============================] - 0s 2ms/step - loss: 5.0363 - accuracy: 0.8402 - val_loss: 4.5126 - val_accuracy: 0.8513
Epoch 6/1000
84/84 [==============================] - 0s 2ms/step - loss: 4.8435 - accuracy: 0.8448 - val_loss: 4.9233 - val_accuracy: 0.8417
Epoch 7/1000
84/84 [==============================] - 0s 2ms/step - loss: 4.7926 - accuracy: 0.8445 - val_loss: 4.6663 - val_accura

### Best Model Advanced
실행 샘플 데이터 - 분포가 많은 값으로 예측한 것으로 추측
-> dropout 을 사용한 결과 확인

In [ ]:
# 최고 모델 사용자 친화적 구현
start_time = time.time()
print("[안내] 모델이 실행됩니다.")
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(0.1))  # Dropout 추가
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))  # Dropout 추가
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))  # Dropout 추가
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.1))  # Dropout 추가
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.1))  # Dropout 추가
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='adam', metrics=[accuracy])

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=5)

if input_2 == 'y':
    model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])
    y_pred = model.predict(X)
else:
    model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
    y_pred = model.predict(X)
    
print("[안내] 최종 모델")
if input_2 == 'y':
    print("[안내] train loss, accuracy")
    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
    print("[안내] validation loss, accuracy")
    loss, acc = model.evaluate(X_val, y_val, verbose=2)
else:
    print("[안내] train loss, accuracy")
    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
    print("[안내] test loss, accuracy")
    loss, acc = model.evaluate(X_test, y_test, verbose=2)

end_time = time.time()

execution_time = end_time - start_time
print("[안내] 실행 시간 : {:.3f} seconds".format(execution_time))

input_3 = input("[안내] 예측 샘플을 확인할까요? : ")
if input_3 == 'y':
    print("[안내] 샘플 10개의 결과")
    new_y = y
    stacked_array = np.vstack((y_pred))
    new_df = pd.DataFrame(stacked_array)
    new_y = pd.DataFrame(new_y)
    new_y['pred'] = new_df[0]
    print(new_y.sample(10))
else:
    print("[안내] 실행을 종료합니다.")

[안내] 모델이 실행됩니다.
Epoch 1/1000
84/84 [==============================] - 2s 4ms/step - loss: 39.9279 - accuracy: 0.4857 - val_loss: 8.6186 - val_accuracy: 0.8031
Epoch 2/1000
84/84 [==============================] - 0s 2ms/step - loss: 20.1747 - accuracy: 0.6566 - val_loss: 6.0916 - val_accuracy: 0.8434
Epoch 3/1000
84/84 [==============================] - 0s 2ms/step - loss: 17.6490 - accuracy: 0.6749 - val_loss: 6.9548 - val_accuracy: 0.8296
Epoch 4/1000
84/84 [==============================] - 0s 2ms/step - loss: 16.1590 - accuracy: 0.6954 - val_loss: 6.1991 - val_accuracy: 0.8353
Epoch 5/1000
84/84 [==============================] - 0s 2ms/step - loss: 14.0945 - accuracy: 0.7142 - val_loss: 5.5175 - val_accuracy: 0.8501
Epoch 6/1000
84/84 [==============================] - 0s 2ms/step - loss: 13.4944 - accuracy: 0.7231 - val_loss: 5.3725 - val_accuracy: 0.8489
Epoch 7/1000
84/84 [==============================] - 0s 2ms/step - loss: 13.3775 - accuracy: 0.7276 - val_loss: 4.9858 - val_

### 최종 model 구현
* 고려사항

1. 실행시간이 짧은 Best Model 1번 사용

2. 사용자 친화적 구성

In [ ]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
def create_best():
    print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
    input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
    input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")

    # 표준화 진행 여부
    if input_1 == 'y':
        scaler = StandardScaler()
        X = scaler.fit_transform(x)
        print("[안내] 데이터 표준화를 진행했습니다.")
    else:
        X = x
        print("[안내] 데이터 표준화를 진행하지 않습니다.")

    # 검증 데이터 진행 여부
    if input_2 == 'y':
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        print("[안내] 검증 데이터를 추가로 분리했습니다.")

    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        print("[안내] 검증 데이터를 분리하지 않았습니다.")

    # 최고 모델 사용자 친화적 구현
    start_time = time.time()
    print("[안내] 모델이 실행됩니다.")
    model = Sequential()
    model.add(Dense(64, activation='tanh', input_dim=x.shape[1]))
    model.add(Dense(32, activation='tanh'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(8, activation='tanh'))
    model.add(Dense(4, activation='tanh'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer='sgd', metrics=[accuracy])
    early_stopping = EarlyStopping(monitor='accuracy', patience=5)

    if input_2 == 'y':
        model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])
        y_pred = model.predict(X)
    else:
        model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
        y_pred = model.predict(X)

    print("[안내] 최종 모델")
    if input_2 == 'y':
        print("[안내] train loss, accuracy")
        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
        print("[안내] validation loss, accuracy")
        loss, acc = model.evaluate(X_val, y_val, verbose=2)
    else:
        print("[안내] train loss, accuracy")
        train_loss, train_acc = model.evaluate(X_train, y_train, verbose=2)
        print("[안내] test loss, accuracy")
        loss, acc = model.evaluate(X_test, y_test, verbose=2)

    end_time = time.time()

    execution_time = end_time - start_time
    print("[안내] 실행 시간 : {:.3f} seconds".format(execution_time))

    input_3 = input("[안내] 예측 샘플을 확인할까요? : ")
    if input_3 == 'y':
        print("[안내] 샘플 10개의 결과")
        new_y = y
        stacked_array = np.vstack((y_pred))
        new_df = pd.DataFrame(stacked_array)
        new_y = pd.DataFrame(new_y)
        new_y['pred'] = new_df[0]
        print(new_y.sample(10))
        print("[안내] 실행을 종료합니다.")

    else:
        print("[안내] 실행을 종료합니다.")



### 실행

In [ ]:
create_best()

[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.
[안내] 모델이 실행됩니다.
Epoch 1/1000
84/84 [==============================] - 1s 4ms/step - loss: 14.8807 - accuracy: 0.7425 - val_loss: 6.8556 - val_accuracy: 0.7796
Epoch 2/1000
84/84 [==============================] - 0s 2ms/step - loss: 6.4742 - accuracy: 0.8189 - val_loss: 6.0546 - val_accuracy: 0.8004
Epoch 3/1000
84/84 [==============================] - 0s 3ms/step - loss: 5.4681 - accuracy: 0.8347 - val_loss: 5.6419 - val_accuracy: 0.7908
Epoch 4/1000
84/84 [==============================] - 0s 2ms/step - loss: 5.1346 - accuracy: 0.8383 - val_loss: 5.2808 - val_accuracy: 0.8156
Epoch 5/1000
84/84 [==============================] - 0s 2ms/step - loss: 5.0836 - accuracy: 0.8383 - val_loss: 4.9870 - val_accuracy: 0.8370
Epoch 6/1000
84/84 [==============================] - 0s 2ms/step - loss: 4.8500 - accuracy: 0.8452 - val_loss: 5.0930 - val_accuracy: 0.8229
Epoch 7/1000
84/84 [=============================